In [1]:
cd ../..

/Users/fm/Desktop/Work/Patat


In [2]:
import os
import re
import importlib

In [3]:
import pandas as pd

# Chargement du corpus complet

In [7]:
corpus_existing_filename = 'data/corpus/220908-CorpusNews.csv'

In [8]:
corpus_new_filename = 'data/corpus/220908-CorpusNews.csv'

In [9]:
df_corpus = pd.read_csv(corpus_existing_filename)

In [10]:
df_corpus['site'].value_counts()

www.francesoir.fr              22672
www.francetvinfo.fr            13828
www.breizh-info.com             5471
www.dreuz.info                  5307
www.legorafi.fr                 4538
www.profession-gendarme.com     3743
www.wikistrike.com              3699
www.lelibrepenseur.org          3298
fr.sott.net                     2878
actu.fr                         2801
lezarceleurs.blogspot.com       2366
lesmoutonsrebelles.com          2303
lemediaen442.fr                 2241
www.cnews.fr                    1330
reseauinternational.net         1131
theconversation.com              956
lesakerfrancophone.fr            851
fr.novopress.info                760
www.polemia.com                  616
infodujour.fr                    490
lesobservateurs.ch               365
www.lesalonbeige.fr              345
lesdeqodeurs.fr                   73
Name: site, dtype: int64

# Initialisations

In [ ]:
import patat.imp.importer
import patat.imp.crawler
import patat.imp.parser

In [ ]:
importlib.reload(patat.imp.parser_params)
importlib.reload(patat.imp.crawler)
importlib.reload(patat.imp.parser)

In [ ]:
imp = patat.imp.importer.Importer()
parser = patat.imp.parser.Parser()
crawler = patat.imp.crawler.Crawler()

In [ ]:
importlib.reload(patat.imp.importer)
imp = patat.imp.importer.Importer()

# 1 - Crawling

In [ ]:
list(parser._css_classes.keys())

In [ ]:
sites = ['www.francesoir.fr',
 'www.breizh-info.com',
 'www.lelibrepenseur.org',
 'lesdeqodeurs.fr',
 'lezarceleurs.blogspot.com',
 'www.dreuz.info',
 'lesmoutonsrebelles.com',
 'www.wikistrike.com',
 'fr.sott.net',
 'lemediaen442.fr',
 'www.profession-gendarme.com',
 'www.legorafi.fr',
 'theconversation.com',
 'actu.fr',
 'www.francetvinfo.fr',
 'www.voltairenet.org',
 'www.mondialisation.ca',
 'www.polemia.com',
 'infodujour.fr',
 'www.lesalonbeige.fr',
 'www.les-crises.fr',
 'www.cnews.fr',
 'lalettrepatriote.com',
 'lesakerfrancophone.fr',
 'lesobservateurs.ch',
 'fr.novopress.info',
 'reseauinternational.net']

In [ ]:
sites = ['www.francesoir.fr',
 'www.breizh-info.com',
 'www.lelibrepenseur.org',
 'lesdeqodeurs.fr',
 'lezarceleurs.blogspot.com',
 'www.dreuz.info',
 'lesmoutonsrebelles.com',
 'www.wikistrike.com',
 'fr.sott.net',
 'lemediaen442.fr',
 'www.profession-gendarme.com',
 'www.legorafi.fr',
 'theconversation.com',
 'actu.fr',
 'www.francetvinfo.fr',
 'www.voltairenet.org',
 'www.mondialisation.ca',
 'www.polemia.com',
 'infodujour.fr',
 'www.lesalonbeige.fr',
 'www.les-crises.fr',
 'www.cnews.fr',
 'lalettrepatriote.com',
 'lesakerfrancophone.fr',
 'lesobservateurs.ch',
 'fr.novopress.info',
 'reseauinternational.net']

In [ ]:
for site in sites:
    url = 'https://'+site
    crawler.crawl_page(url, use_cache=False)

In [ ]:
len(crawler._crawl_map['list_links'])

In [ ]:
len(crawler._crawl_map['article_links'])

In [ ]:
crawler.crawl_links(crawler._crawl_map['list_links'],max=1000,use_cache=False)

## Import et Parsing

In [ ]:
%%time
n = 1
pages = {}
parsed_articles = []
for url in crawler._crawl_map['article_links']:
    print(str(n)+' '+url)
    if imp.page_in_cache(url)==False:
        pages[url] = imp.get_webpage(url)
        if pages[url]:
            parsed_articles.append(parser.parse(url,pages[url]))
    n+=1


In [ ]:
len(parsed_articles)

In [ ]:
df_new = pd.DataFrame(parsed_articles)

In [ ]:
import urllib
def get_site(url):
    return urllib.parse.urlsplit(url).netloc

In [ ]:
df_new.loc[:,'site'] = df_new['url'].apply(get_site)

In [ ]:
counts = df_new['site'].value_counts()

In [ ]:
counts

In [ ]:
df_new

## Merge and save

In [ ]:
len(df_corpus)

In [ ]:
df_merge = pd.concat([df_new,df_corpus],axis=0)

In [ ]:
df_merge['site'].value_counts()

In [ ]:
len(df_merge)

In [ ]:
df_merge = df_merge.drop_duplicates(subset=['url'])

In [ ]:
df_merge['site'].value_counts()

In [ ]:
df_merge = df_merge.drop(['article_links', 'list_links', 'local_links', 'external_links',],axis=1)

In [ ]:
df_merge = df_merge[df_merge['article'].notna()]

In [ ]:
df_merge.to_csv(corpus_new_filename,index=False)

In [ ]:
df_merge.loc[:,'date_iso'] = df_merge['date_iso'].fillna('')

# Lecture du cache

In [ ]:
sites

In [ ]:
html_files = {}
pages = {}
for site in sites:
    html_files[site] = os.listdir('data/html/'+site)
    pages[site] = []
    for filename in html_files[site]:
        pages[site].append(re.sub(r'\|','/','https://'+site+filename))
        

In [ ]:
article_urls = {}
for site in pages:
    article_urls[site]=[]
    for page in pages[site]:
        if parser.is_article_url(page,site):
            article_urls[site].append(page)

In [ ]:
len(article_urls)

In [ ]:
article_urls

In [ ]:
i = 1
parsed_articles = []
for site in article_urls:
    for article_url in article_urls[site]:
        print(i,article_url)
        html = imp.get_webpage(article_url)
        parsed_articles.append(parser.parse_article(article_url,html,site))
        i = i+1

In [ ]:
df_new = pd.DataFrame(parsed_articles)

In [ ]:
df_new

In [ ]:
import urllib
def get_site(url):
    return urllib.parse.urlsplit(url).netloc

In [ ]:
df_new.loc[:,'site'] = df_corpus['url'].apply(get_site)

In [ ]:
df_new['site'].value_counts()

## Merge Corpus

In [ ]:
df_merge = pd.concat([df_new,df_corpus],axis=0)

In [ ]:
df_merge['site'].value_counts()

In [ ]:
df_merge = df_merge.drop_duplicates(subset=['url'])

In [ ]:
df_merge['site'].value_counts()

In [ ]:
df_merge.to_csv(corpus_new_filename,index=False)

# Bricolages

In [ ]:
parser.is_article_url('https://theconversation.com/institutions/graduate-institute-institut-de-hautes-etudes-internationales-et-du-developpement-iheid-2905','theconversation.com')

In [ ]:
html = imp.get_webpage('https://theconversation.com')

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(html,parser='lxml')

In [ ]:
for a in soup.find_all('a'):
    print(a.get('href'))

In [ ]:
parser.parse('https://fr.novopress.info',html,'fr.novopress.info')

In [ ]:
parse_data = parser.parse('https://theconversation.com',html)

In [ ]:
parse_data['list_links']

In [ ]:
for article in df_corpus[df_corpus['site']=='actu.fr']['article']:
    print(article)

In [ ]:
df_corpus.loc[:,'date_iso']=df_corpus.apply(lambda row : parser.text_to_date(row['date'],row['site']),axis=1)

In [ ]:
df_corpus['date'].fillna('',inplace=True)

In [ ]:
df_corpus

In [ ]:
df_corpus.to_csv(corpus_new_filename,index=False)